In [1]:
import numpy as np
from scipy.stats import truncnorm
from simulation_utils import *

In [5]:
gamma_a = 1
gamma_n = 1/2.0
delta_a = 5
delta_n = 5
num_time_steps = 15
num_units= 30
window_size = 5
treatment_dosage = 1
drug_half_life = 1

global_min = 0.1
global_max = 10


K = 10
rho = -0.001
beta_c = 0.03
beta_n = beta_c/3.0

ratio = 1.0
dataset = "Flickr"

In [6]:
def sigmod(x):
    return 1/(1 + np.exp(-x))

In [7]:
data,parts = read_data_debug(dataset)
train_index,val_index,test_index = data_split(parts)
train_A, val_A, test_A = adj_split(data,train_index,val_index,test_index,dataset)

Size of train graph:1591, val graph:2526, test graph:1763
<class 'numpy.ndarray'>
Shape of adj matrix train:(1591, 1591), val:(2526, 2526), test:(1763, 1763)


In [ ]:
num_units = train_A.shape[0]
adj = train_A

In [ ]:
# Patient cancer stage. (mu, sigma, lower bound, upper bound) - for lognormal dist
health_stages_stats = {
    "I": (1.5, 1, global_min, 3),
    "II": (4.5, 1, 3, 6),
    "III": (8, 1, 6, global_max),
}
possible_stages = list(health_stages_stats.keys())
possible_stages.sort()
initial_stages = np.random.choice(possible_stages,num_units)

initial_health = []
for i in range(num_units):
    stg = initial_stages[i]
    mu, sigma, lower_bound, upper_bound = health_stages_stats[stg]
    initial_health_by_stage = np.random.normal(mu, sigma, 1)[0]
    if initial_health_by_stage<lower_bound:
        initial_health_by_stage = lower_bound
    elif initial_health_by_stage>upper_bound:
        initial_health_by_stage = upper_bound
    initial_health.append(initial_health_by_stage)


health_condition = np.zeros((num_units, num_time_steps))
health_condition[:,0] = initial_health

cf_health_condition = np.zeros((num_units, num_time_steps))
cf_health_condition[:,0] = initial_health


In [ ]:
treatment_probs = np.zeros((num_units, num_time_steps))
treatment_application_rvs = np.random.rand(num_units, num_time_steps)

action_application_point = np.zeros((num_units, num_time_steps))
avg_neighbor_action_application = np.zeros((num_units, num_time_steps))
action_dosage = np.zeros((num_units, num_time_steps))
avg_neighbor_action_dosage = np.zeros((num_units, num_time_steps))

avg_health_used_for_all = np.zeros((num_units, num_time_steps))
avg_neighbor_health_used_for_all = np.zeros((num_units, num_time_steps))

noise_terms = 0.01 * np.random.randn(num_units,num_time_steps) 
for t in range(0, num_time_steps - 1):
    for i in range(num_units):
        health_used = health_condition[i, max(t - window_size, 0) : t + 1]
        avg_health_used_for_all[i][t] = np.array([health for health in health_used]).mean()
    temp = np.matmul(adj,avg_health_used_for_all[:,t])
    nei_sum = np.sum(adj,1)
    avg_neighbor_health_t = np.divide(temp,nei_sum)
    avg_neighbor_health_used_for_all[:,t] = avg_neighbor_health_t

    for i in range(num_units):
        current_dose = 0.0
        previous_dose = 0.0 if t == 0 else action_dosage[i, t - 1]

        health_used = health_condition[i, max(t - window_size, 0) : t + 1]
        avg_health_used = np.array([health for health in health_used]).mean()
        avg_neighbor_health_used = avg_neighbor_health_used_for_all[i][t]

        treatment_prob_t = sigmod(-1*(gamma_a*(avg_health_used-delta_a)+gamma_n*(avg_neighbor_health_used-delta_n)))
        treatment_probs[i, t] = treatment_prob_t

        if treatment_application_rvs[i, t] < treatment_prob_t:
            action_application_point[i, t] = 1
            current_dose = treatment_dosage

        # Update chemo dosage
        action_dosage[i, t] = previous_dose * np.exp(-np.log(2) / drug_half_life)+ current_dose

    for i in range(num_units):
        temp = np.matmul(adj,action_dosage[:,t])
        nei_sum = np.sum(adj,1)
        avg_neighbor_dosage = np.divide(temp,nei_sum)
        avg_neighbor_action_dosage[:,t] = avg_neighbor_dosage

    for i in range(num_units):
        temp = np.matmul(adj,action_application_point[:,t])
        nei_sum = np.sum(adj,1)
        avg_neighbor_treatment = np.divide(temp,nei_sum)
        avg_neighbor_action_application[:,t] = avg_neighbor_treatment

    for i in range(num_units):

        health_condition[i, t + 1] = health_condition[i, t] * (
            1
            +rho * np.log10(K / health_condition[i, t])
            + beta_c * action_dosage[i, t]
            + (beta_n * avg_neighbor_action_dosage[i, t])
            + noise_terms[i][t]
        ) 

        if  health_condition[i, t + 1] > global_max:
            health_condition[i, t + 1] = global_max
        if  health_condition[i, t + 1] < global_min:
            health_condition[i, t + 1] = global_min

In [ ]:
cf_action_application_point = np.zeros((num_units, num_time_steps))
cf_avg_neighbor_action_application = np.zeros((num_units, num_time_steps))


r,c = action_application_point.shape
flip_mask = 1.0*(np.random.rand(r,c)<=ratio)
for ii in range(r):
    for jj in range(c):
        if flip_mask[ii][jj]==1:
            cf_action_application_point[ii][jj] = action_application_point[ii][jj]*-1+1
        else:
            cf_action_application_point[ii][jj] = action_application_point[ii][jj]


cf_action_dosage = np.zeros((num_units, num_time_steps))
cf_avg_neighbor_action_dosage = np.zeros((num_units, num_time_steps))

cf_noise_terms = 0.01 * np.random.randn(num_units,num_time_steps) 

for t in range(0, num_time_steps - 1):


    for i in range(num_units):
        current_dose = 0.0
        previous_dose = 0.0 if t == 0 else cf_action_dosage[i, t - 1]

        if cf_action_application_point[i, t] ==1:
            current_dose = treatment_dosage

        # Update chemo dosage
        cf_action_dosage[i, t] = previous_dose * np.exp(-np.log(2) / drug_half_life)+ current_dose

    for i in range(num_units):
        temp = np.matmul(adj,cf_action_dosage[:,t])
        nei_sum = np.sum(adj,1)
        cf_avg_neighbor_dosage = np.divide(temp,nei_sum)
        cf_avg_neighbor_action_dosage[:,t] = cf_avg_neighbor_dosage

    for i in range(num_units):
        temp = np.matmul(adj,cf_action_application_point[:,t])
        nei_sum = np.sum(adj,1)
        cf_avg_neighbor_treatment = np.divide(temp,nei_sum)
        cf_avg_neighbor_action_application[:,t] = cf_avg_neighbor_treatment


    for i in range(num_units):

        cf_health_condition[i, t + 1] = cf_health_condition[i, t] * (
            1
            +rho * np.log10(K / cf_health_condition[i, t])
            + beta_c * cf_action_dosage[i, t]
            + (beta_n * cf_avg_neighbor_action_dosage[i, t])
            + cf_noise_terms[i][t]
        ) 

        if  cf_health_condition[i, t + 1] > global_max:
            cf_health_condition[i, t + 1] = global_max
        if  cf_health_condition[i, t + 1] < global_min:
            cf_health_condition[i, t + 1] = global_min

In [ ]:
outputs = {
    "health_condition": health_condition,
    "action_dosage": action_dosage,
    "avg_neighbor_action_dosage": avg_neighbor_action_dosage,
    "action_application_point": action_application_point,
    "treatment_probs": treatment_probs,
    "noise_terms":noise_terms,
    "cf_health_condition": cf_health_condition,
    "cf_action_application_point":cf_action_application_point,
    "cf_avg_neighbor_action_application":cf_avg_neighbor_action_application,
}

In [ ]:
def get_scaling_params(sim):
    real_idx = ["health_condition", "action_dosage", "avg_neighbor_action_dosage"]

    means = {}
    stds = {}
    for k in real_idx:
        means[k] = np.mean(sim[k])
        stds[k] = np.std(sim[k])
    
    sim["mean"] = means
    sim["std"] = stds

    return sim

In [ ]:
outputs = get_scaling_params(outputs)
save_sim_debug(dataset,outputs)

In [ ]:
outputs["mean"].keys()

In [ ]:
np.mean(np.abs(action_application_point-cf_action_application_point))

In [ ]:
np.mean(cf_health_condition[:,0]-health_condition[:,0])

In [ ]:
np.mean(np.abs(cf_health_condition-health_condition)**2)

In [ ]:
avg_neighbor_health_used_for_all

In [ ]:
treatment_application_rvs

In [ ]:
treatment_probs

In [ ]:
action_dosage

In [ ]:
avg_neighbor_action_dosage

In [ ]:
health_condition

In [ ]:
cf_health_condition